# Use case scenario 1

One possible use case for this data would be to create maps of predicted maximum temperature accross the county for various future dates. An overall understanding of possible future temperatures can create urgency for county and city officials to identify vulnerable populations such as low income households and implement programs to address these vulnerabilities. These maps would also provide officials with the information necessary to plan and allocate resources based on the need in specific areas. For instance, if certain areas of the county are predicted to experience life threatening heat, preparations can be made to protect residents of those areas and implement mitigation measures such as adequate air conditioning. Other risk factors in these areas can also be identified and addressed, such as possible fire risks. Additionally, these maps could aid in the understanding of economic effects of future temperatures such as possible changes in crop yields.

# Use case scenario 2

Another use case scenario for this data would be to create maps of predicted precipitation across the county. Possible future changes in precipitation have serious implications for water resources planning and disaster mitigation. Increased precipitation could lead to an increase in flooding, landslides, and debris flows. Decreased precipitation can lead to drought, increased fire risk, and negative economic effects due to the effect on crops and livestock. These maps could provide county and city officials with the information needed to plan for possible changes in precipitation in various areas accross the county. 